In [1]:
import cv2
cap = cv2.VideoCapture(0)
face_model = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
import numpy as np
count = 0
def face_extractor(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_model.detectMultiScale(gray)
    
    if len(faces) == 0:
        return None
    for x,y,w,h in faces:
        cropped_face = img[y:y+h,x:x+w]
        
    return cropped_face



while True:
    status , photo = cap.read()
    if face_extractor(photo) is not None:
        count += 1
        face = cv2.resize(face_extractor(photo),(224,224))
        
    
        filename_path = "C://Users//Sahiba//Desktop//mlops-ws//Task1//Images//Validation//image"+str(count)+'.jpg'
        cv2.imwrite(filename_path,face)
        
        
        cv2.putText(face,str(count),(10,20),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
        cv2.imshow("FaceCrop",face)
        
    else:
        print("Face Not Found")
        pass
    
    if cv2.waitKey(10) == 13 or count == 100:
        break
        
        
cv2.destroyAllWindows()
cap.release()
print("Collected samples completed")

Collected samples completed


In [31]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
#img_rows = 224
#img_cols = 224 

# Re-loads the VGG16 model without the top or FC layers
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (224, 224, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [3]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dense(128,activation = "relu")(top_model)
    top_model = Dense(64,activation = "relu")(top_model)
    top_model = Dense(32,activation = "relu")(top_model)
    top_model = Dense(10,activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Images/Training'
validation_data_dir = 'Images/Testing'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 200 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [6]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [7]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("face_detection_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 3
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save("face_detection_vgg.h5")

Epoch 1/3
74/74 [==============================] - 314s 4s/step - loss: 0.1845 - accuracy: 0.9038 - val_loss: 2.8800e-05 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.00003, saving model to face_detection_vgg.h5
Epoch 2/3
74/74 [==============================] - 312s 4s/step - loss: 0.0526 - accuracy: 0.9850 - val_loss: 0.0041 - val_accuracy: 1.0000

Epoch 00002: val_loss did not improve from 0.00003
Epoch 3/3
74/74 [==============================] - 312s 4s/step - loss: 0.0341 - accuracy: 0.9982 - val_loss: 1.8239e-06 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.00003 to 0.00000, saving model to face_detection_vgg.h5


In [8]:
from keras.models import load_model

In [9]:
classifier = load_model('face_detection_vgg.h5')

In [12]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

monkey_breeds_dict = {"[0]": "Gaurav", 
                      "[1]": "Sahiba"  
                     }

monkey_breeds_dict_n = {"Gaurav": "Gaurav",
                        "Sahiba": "Sahiba"
                       }

def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    
    random_directory = np.random.randint(0,len(folders))
    
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("Images/Testing/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - Gaurav
Class - Gaurav
Class - Sahiba
Class - Gaurav
Class - Gaurav
Class - Gaurav
Class - Gaurav
Class - Sahiba
Class - Sahiba
Class - Sahiba


In [32]:
from keras.models import load_model

In [33]:
model=load_model("face_detection_vgg.h5")

In [34]:
import numpy as np

import os
#import IPython.display as ipd
#import webbrowser as wb
import cv2
from keras.preprocessing import image
import PIL

In [35]:
face_model = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [36]:
def face_detector(img ,size=0.5):
    faces = face_model.detectMultiScale(img, 1.3 ,5)
    if faces is ():
        return img, []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0 ,255, 255),2)
        roi = img[y:y+h,x:x+w]
        roi = cv2.resize(roi, (224, 224))
    return img, roi

In [37]:
cap = cv2.VideoCapture(0)

while True:

    status, photo = cap.read()
    image, face = face_detector(photo)
    try:
        face=np.array(face)
        face=np.expand_dims(face, axis=0)
        if(face.shape==(1,0)):
            cv2.putText(image, "Not Detected", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition',image)
            print("3")
            pass
        else:
            results=model.predict(face)
            if(results[0][0] == 0.0):
                cv2.putText(image, "Gaurav", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                cv2.imshow('Face Recognition',image)
                print("1")
            elif(results[0][0] != 0.0):
                cv2.putText(image, "Sahiba", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                cv2.imshow('Face Recognition',image)
                print("2")
                
    except:
        cv2.imshow('Face Recognition', image )
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        print("4")
        pass
    if  cv2.waitKey(1) == 13: #13 is the Enter Key
        break
cap.release()
cv2.destroyAllWindows()

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
3
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
